In [1]:
# import libraries
from gettext import npgettext
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import numpy as np
from statistics import mean, stdev

import json

In [2]:
# setup spark
spark = (
    SparkSession.builder.appName("aggregate data for first 3 final model variables")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "15g")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/28 16:02:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/28 16:02:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/28 16:02:34 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
postcode_df = spark.read.csv("../data/curated/Grouping.csv", header = "true")
persona_df = spark.read.csv("../data/curated/abs_cleaned.csv",  header = "true")

In [4]:
persona_df = persona_df.withColumnRenamed("_c0","postcode")

In [5]:
# mean age
cond = ["postcode"]
total_df = postcode_df.join(persona_df,cond, "left")

In [6]:
total_df

22/09/28 16:02:38 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/09/28 16:02:38 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , mean_Age, min_Age, q1_Age, median_Age, q3_Age, max_Age, Millen_Z%, Oceania and Antarctica, North-West Europe, Southern and Eastern Europe, North Africa and the Middle East, South-East Asia, North-East Asia, Southern and Central Asia, Americas, Sub-Saharan Africa, mean_Salary, median_Salary, min_Salary, q1_Salary, q3_Salary, max_Salary, Managers_%, Professionals_%, Technicians and Trades Workers_%, Community and Personal Service Workers_%, Clerical and Administrative Workers_%, Sales Workers_%, Machinery Operators and Drivers_%, Labourers_%, Total
 Schema: _c0, mean_Age, min_Age, q1_Age, median_Age, q3_Age, max_Age, Millen_Z%, Oceania and Antarctica, North-West Europe, Southern and Eastern Europe, North Africa and th

postcode,merchant_abn,Fortnight,count_postcode_merchant,total_order_per_fornight,proportion_of_postcode,mean_Age,min_Age,q1_Age,median_Age,q3_Age,max_Age,Millen_Z%,Oceania and Antarctica,North-West Europe,Southern and Eastern Europe,North Africa and the Middle East,South-East Asia,North-East Asia,Southern and Central Asia,Americas,Sub-Saharan Africa,mean_Salary,median_Salary,min_Salary,q1_Salary,q3_Salary,max_Salary,Managers_%,Professionals_%,Technicians and Trades Workers_%,Community and Personal Service Workers_%,Clerical and Administrative Workers_%,Sales Workers_%,Machinery Operators and Drivers_%,Labourers_%,Total
6068,10206519221,1,1,151,0.006622516556291391,70.0,65,65,65,65,65,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,385.71,900.0,900.0,900.0,900.0,900.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7142857142857143,7
2565,10206519221,1,1,151,0.006622516556291391,40.840889734983215,0,18,34,53,98,0.33239817644406594,0.6519967837041007,0.043634414366121685,0.030715625837577056,0.02106673813990887,0.06432591798445457,0.0282497989815063,0.13063521844009648,0.01618868935942107,0.013186813186813187,587.7,725.0,0.0,350.0,1125.0,1625.0,0.0436327245434075,0.0811608706529897,0.0586439829872404,0.047035276457343,0.0791093319989992,0.0411808856642481,0.0482862146609957,0.0459844883662747,19985
6411,10206519221,1,1,151,0.006622516556291391,54.4040404040404,0,40,53,61,80,0.15151515151515152,0.8545454545454545,0.11818181818181818,0.0,0.0,0.02727272727272727,0.0,0.0,0.0,0.0,797.98,725.0,225.0,450.0,1375.0,2500.0,0.2177419354838709,0.0564516129032258,0.064516129032258,0.0241935483870967,0.0564516129032258,0.032258064516129,0.032258064516129,0.064516129032258,124
5372,10206519221,1,1,151,0.006622516556291391,41.28984198645598,0,15,35,54,100,0.2984198645598194,0.8969813128893148,0.08864398658361285,0.003354096789650...,0.0,0.005749880210828941,0.0,0.002395783421178...,0.002874940105414...,0.0,547.88,725.0,75.0,350.0,1125.0,1625.0,0.0573881608676005,0.0460912788070492,0.0759150474469046,0.0632625395390872,0.0564844103027564,0.0375056484410302,0.0551287844554902,0.0718481699051061,2213
6438,10206519221,1,1,151,0.006622516556291391,41.9400479616307,0,25,36,51,90,0.3657074340527578,0.8642745709828393,0.06396255850234009,0.0109204368174727,0.0,0.028081123244929798,0.0046801872074883,0.0,0.0062402496099844,0.0218408736349454,892.42,1125.0,225.0,450.0,1875.0,2500.0,0.0496969696969697,0.0678787878787878,0.0933333333333333,0.0387878787878787,0.0387878787878787,0.0193939393939393,0.1212121212121212,0.0472727272727272,825
2753,10206519221,1,1,151,0.006622516556291391,44.00262574396079,0,19,39,57,98,0.30657019488855175,0.8664407205171656,0.0630138705830666,0.031318646833615765,0.003891294796962...,0.011360070294357623,0.006778384485031...,0.004895499905855771,0.005711416556831733,0.006590096027113538,625.49,725.0,75.0,350.0,1125.0,1875.0,0.0573713085093965,0.0732461771915489,0.0877786856542547,0.0516516867047974,0.0666511030699194,0.0444729777051476,0.0442978872417415,0.0495506011439243,17134
4072,10206519221,1,1,151,0.006622516556291391,24.91883116883117,12,18,19,20,68,0.9828385899814471,0.7432751297781972,0.04341670599339311,0.003303445021236...,0.005663048607833884,0.06795658329400661,0.06182161396885323,0.018876828692779613,0.03067484662576687,0.025011798017932987,156.73,75.0,0.0,0.0,225.0,350.0,0.0102183000464468,0.048769159312587,0.0051091500232234,0.0975383186251741,0.0404087320018578,0.0729215048769159,0.0018578727357176,0.039479795633999,2153
4673,10206519221,1,1,151,0.006622516556291391,49.590634441087616,0,23,51,63,84,0.19939577039274925,0.9208754208754208,0.06060606060606061,0.006734006734006734,0.0,0.005050505050505051,0.0,0.0,0.006734006734006734,0.0,499.96,450.0,75.0,225.0,900.0,1375.0,0.0770416024653312,0.0338983050847457,0.049306625577812,0.0338983050847457,0.0215716486902927,0.0200308166409861,0.0416024653312788,0.0631741140215716,649
3920,10206519221,1,1,151,0.006622516556291391,28.290753098188752,0,19,21,27,59,0.8217349857006673,0.9198813056379822,

In [7]:
listofmult = total_df.columns[6:]
listofmult

['mean_Age',
 'min_Age',
 'q1_Age',
 'median_Age',
 'q3_Age',
 'max_Age',
 'Millen_Z%',
 'Oceania and Antarctica',
 'North-West Europe',
 'Southern and Eastern Europe',
 'North Africa and the Middle East',
 'South-East Asia',
 'North-East Asia',
 'Southern and Central Asia',
 'Americas',
 'Sub-Saharan Africa',
 'mean_Salary',
 'median_Salary',
 'min_Salary',
 'q1_Salary',
 'q3_Salary',
 'max_Salary',
 'Managers_%',
 'Professionals_%',
 'Technicians and Trades Workers_%',
 'Community and Personal Service Workers_%',
 'Clerical and Administrative Workers_%',
 'Sales Workers_%',
 'Machinery Operators and Drivers_%',
 'Labourers_%',
 'Total']

In [8]:
total_df = total_df.withColumn("arr", F.struct(*[(F.col(x)*F.col('proportion_of_postcode')).alias(x) for x in listofmult]))\
  .select("postcode","merchant_abn","Fortnight","count_postcode_merchant","total_order_per_fornight","arr.*")

In [9]:
exprs = {x: "sum" for x in listofmult}
final_df1 = total_df.groupBy("merchant_abn","Fortnight").agg(exprs)

In [10]:
final_df1.toPandas().to_csv('../data/curated/Postcode_Persona.csv', index = False)

22/09/28 16:02:40 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , mean_Age, min_Age, q1_Age, median_Age, q3_Age, max_Age, Millen_Z%, Oceania and Antarctica, North-West Europe, Southern and Eastern Europe, North Africa and the Middle East, South-East Asia, North-East Asia, Southern and Central Asia, Americas, Sub-Saharan Africa, mean_Salary, median_Salary, min_Salary, q1_Salary, q3_Salary, max_Salary, Managers_%, Professionals_%, Technicians and Trades Workers_%, Community and Personal Service Workers_%, Clerical and Administrative Workers_%, Sales Workers_%, Machinery Operators and Drivers_%, Labourers_%, Total
 Schema: _c0, mean_Age, min_Age, q1_Age, median_Age, q3_Age, max_Age, Millen_Z%, Oceania and Antarctica, North-West Europe, Southern and Eastern Europe, North Africa and the Middle East, South-East Asia, North-East Asia, Southern and Central Asia, Americas, Sub-Saharan Africa, mean_Salary, median_Salary, min_Salary, q1_Salary, q3_Salary, max_Salary, 

In [12]:
final_df1

22/09/28 16:04:57 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , mean_Age, min_Age, q1_Age, median_Age, q3_Age, max_Age, Millen_Z%, Oceania and Antarctica, North-West Europe, Southern and Eastern Europe, North Africa and the Middle East, South-East Asia, North-East Asia, Southern and Central Asia, Americas, Sub-Saharan Africa, mean_Salary, median_Salary, min_Salary, q1_Salary, q3_Salary, max_Salary, Managers_%, Professionals_%, Technicians and Trades Workers_%, Community and Personal Service Workers_%, Clerical and Administrative Workers_%, Sales Workers_%, Machinery Operators and Drivers_%, Labourers_%, Total
 Schema: _c0, mean_Age, min_Age, q1_Age, median_Age, q3_Age, max_Age, Millen_Z%, Oceania and Antarctica, North-West Europe, Southern and Eastern Europe, North Africa and the Middle East, South-East Asia, North-East Asia, Southern and Central Asia, Americas, Sub-Saharan Africa, mean_Salary, median_Salary, min_Salary, q1_Salary, q3_Salary, max_Salary, 

22/09/28 16:05:47 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , mean_Age, min_Age, q1_Age, median_Age, q3_Age, max_Age, Millen_Z%, Oceania and Antarctica, North-West Europe, Southern and Eastern Europe, North Africa and the Middle East, South-East Asia, North-East Asia, Southern and Central Asia, Americas, Sub-Saharan Africa, mean_Salary, median_Salary, min_Salary, q1_Salary, q3_Salary, max_Salary, Managers_%, Professionals_%, Technicians and Trades Workers_%, Community and Personal Service Workers_%, Clerical and Administrative Workers_%, Sales Workers_%, Machinery Operators and Drivers_%, Labourers_%, Total
 Schema: _c0, mean_Age, min_Age, q1_Age, median_Age, q3_Age, max_Age, Millen_Z%, Oceania and Antarctica, North-West Europe, Southern and Eastern Europe, North Africa and the Middle East, South-East Asia, North-East Asia, Southern and Central Asia, Americas, Sub-Saharan Africa, mean_Salary, median_Salary, min_Salary, q1_Salary, q3_Salary, max_Salary, 

merchant_abn,Fortnight,sum(Professionals_%),sum(median_Age),sum(q1_Salary),sum(Machinery Operators and Drivers_%),sum(Southern and Central Asia),sum(max_Age),sum(min_Salary),sum(Oceania and Antarctica),sum(North-East Asia),sum(Sales Workers_%),sum(mean_Salary),sum(Sub-Saharan Africa),sum(Technicians and Trades Workers_%),sum(Community and Personal Service Workers_%),sum(North Africa and the Middle East),sum(q3_Age),sum(Total),sum(North-West Europe),sum(max_Salary),sum(mean_Age),sum(q1_Age),sum(Labourers_%),sum(Southern and Eastern Europe),sum(min_Age),sum(Managers_%),sum(South-East Asia),sum(Americas),sum(median_Salary),sum(q3_Salary),sum(Clerical and Administrative Workers_%),sum(Millen_Z%)
74449830326,1,0.07687218816590007,40.62962962962962,332.75462962962945,0.025874003928065707,0.011355106034790206,87.40740740740739,72.26851851851848,0.8002164363825824,0.025444047431888946,0.02881707637626296,590.4459259259258,0.011410811967344505,0.0504787870276589,0.04320416670117859,0.006025874429600389,55.09259259259259,7745.648148148147,0.06014190313571046,1758.3333333333335,43.840436949550586,21.38888888888889,0.051299907386434695,0.017609185667672687,0.8888888888888887,0.10664592507680676,0.02274202827636909,0.008017569637003945,633.7962962962963,1158.101851851852,0.04657292971859235,0.26634514816234106
48452300785,2,0.1083124843039702,45.5,334.375,0.016556399422336762,0.011717357799815076,94.0,37.5,0.7988665730783215,0.020334369490991728,0.03550848607719971,710.88,0.01525502907871536,0.04115174200037384,0.04129595373516164,0.007807662888756665,61.375,10443.75,0.08340946887677807,2265.625,47.897573626800934,21.875,0.03372696536930734,0.019871879391429884,0.0,0.0755177858918944,0.02529322554641704,0.017444433848774574,746.875,1434.375,0.05036210005895351,0.24316053417340566
94547309480,2,0.06930371383500974,43.11111111111111,372.2222222222222,0.021352965034774074,0.002392870763833...,85.33333333333333,116.66666666666664,0.8980793238904221,0.005732242053415436,0.03804975947458234,613.2344444444444,0.011085327740500492,0.054879982188722524,0.04393985262374027,0.001535550265552...,58.66666666666666,3420.555555555555,0.05926745509523603,1844.4444444444448,45.46062026769029,19.77777777777778,0.06129404144431094,0.010436447642149575,0.3333333333333333,0.12530827063128067,0.005624913836002...,0.005845868712887664,661.1111111111112,1183.3333333333333,0.0513842756373252,0.23647940483766
50131473008,5,0.049817447342302965,42.800000000000004,325.0,0.03726606505961311,0.01945082962638312,91.80000000000001,60.0,0.840437160402932,0.004864596811845832,0.03643594193242644,529.388,0.004121441066834915,0.0630381883970507,0.04614222687621816,0.019618159920163553,59.400000000000006,7223.400000000001,0.03823849523927545,1525.0,46.05901199813528,20.000000000000004,0.06227314124031175,0.05116314110277276,0.2,0.07994140259693469,0.018678026019767188,0.003428149810025076,555.0,990.0,0.042801088136522385,0.25351838952850286
73221370757,5,0.10270181974843912,44.14285714285714,346.42857142857144,0.02096016839422643,0.01247552079672301,91.28571428571429,80.35714285714286,0.8266000430122471,0.017643530694735093,0.03719365482178018,633.6528571428571,0.007351168888147558,0.05648084977681254,0.04284112600987694,0.004356138191628007,59.785714285714285,6537.214285714285,0.0783183813695349,1839.2857142857142,46.97170366404526,23.357142857142854,0.05307791575279688,0.01633968183969547,0.4285714285714285,0.08243593209351285,0.026298528921626964,0.010617006285661923,694.6428571428571,1237.5,0.05284157477236126,0.2500364486863843
96906278264,5,0.07054002847350985,42.27868852459017,331.4549180327867,0.031021886469293183,0.01242781816031752,90.16393442622946,88.85245901639348,0.8547253366216385,0.009718818769897502,0.033693979424667626,580.7245901639346,0.007264447351793611,0.05393838933406331,0.042353170596866295,0.006140186229361436,58.22950819672133,7658.688524590164,0.06413346067017729,1700.2868852459005,45.613236336336335,21.836065573770497,0.05633297381863418,0.0184360